In [1]:
#Packages:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from camera_matrix import camera_matrix
import time
import seaborn as sns
import matplotlib.patches as mpatches
from skimage.segmentation import felzenszwalb

ModuleNotFoundError: No module named 'numpy'

In [3]:
# Charger la vidéo
#cap0 = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/GX010056.MP4")
#video_capture = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/GX010031.MP4")
video_capture = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/GX010056.MP4")
# Définition de la fonction NDVI
def ndvi(image):
    # Convertir l'image en float
    image = image.astype(float)
    # Calculer l'indice NDVI
    ndvi = (image[:,:,1] - image[:,:,2]) / (image[:,:,1] + image[:,:,2] + 1e-5)
    return ndvi

# Définition de la fonction NDWI
def ndwi(image):
    # Convertir l'image en float
    image = image.astype(float)
    # Calculer l'indice NDWI
    ndwi = (image[:,:,1] - image[:,:,0]) / (image[:,:,1] + image[:,:,0] + 1e-5)
    return ndwi

# Lire la première frame de la vidéo
ret, first_frame = video_capture.read()

# Vérifier si la première frame a été lue avec succès
if not ret:
    print("Impossible de lire la première frame de la vidéo.")
    exit()

# Segmentation de la première frame en utilisant l'algorithme de Felzenszwalb
segments = felzenszwalb(first_frame, scale=100, sigma=0.5, min_size=50)

# Calculer l'indice NDVI et NDWI
ndvi_image = ndvi(first_frame)
ndwi_image = ndwi(first_frame)

# Seuils pour la détection de la rivière et de la berge
ndvi_threshold = 0.1  # Seuil NDVI pour la berge
ndwi_threshold = 0.1  # Seuil NDWI pour la rivière

# Créer des masques pour la rivière et la berge en utilisant les indices
river_mask = np.zeros_like(first_frame)
shore_mask = np.zeros_like(first_frame)

river_mask[ndwi_image > ndwi_threshold] = [255, 0, 0]  # Rouge pour la rivière
shore_mask[ndvi_image > ndvi_threshold] = [0, 255, 0]  # Vert pour la berge

# Appliquer une opération de morphologie pour lisser les zones
river_kernel = np.ones((15, 15), np.uint8)  # Noyau pour la rivière
shore_kernel = np.ones((15, 15), np.uint8)  # Noyau pour la berge

river_mask = cv2.morphologyEx(river_mask, cv2.MORPH_OPEN, river_kernel)
shore_mask = cv2.morphologyEx(shore_mask, cv2.MORPH_OPEN, shore_kernel)

# Fusionner les masques
result = cv2.addWeighted(river_mask, 0.5, shore_mask, 0.5, 0)
# Afficher le résultat
cv2.imshow('River Image', result)

# Attendre jusqu'à ce que l'utilisateur appuie sur une touche
cv2.waitKey(0)

# Libérer la vidéo et fermer les fenêtres
video_capture.release()
cv2.destroyAllWindows()